# 125个经典技术指标择时分析
## 广发证券 2020年1月3日

In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time
import os

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed
import indicatorJ

# 基础函数
import utilsJ

## 策略

### 策略主体

In [2]:
class gf125(bt.Strategy):
    
    params = (
        ('printlog', False),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.data.datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to the "close" line, buyprice and sellprice in the data[0] dataseries
        self.buyprice = None
        self.sellprice = None
        
        # Add indicators
        #self.ER = indicatorJ.ER(self.data)
        #self.DPO = indicatorJ.DPO(self.data)
        self.POS = indicatorJ.POS(self.data)
        #self.TII = indicatorJ.TII(self.data)
        #self.ADTM = indicatorJ.ADTM(self.data)

        # Create buy&sell signals
        #self.ER_buy = bt.And(self.ER.lines.BearPower > 0, self.ER.lines.BearPower(-1) < 0)
        #self.ER_sell = bt.And(self.ER.lines.BullPower < 0, self.ER.lines.BullPower(-1) > 0)
        #self.DPO_buy = bt.And(self.DPO.lines.DPO > 0, self.DPO.lines.DPO(-1) < 0)
        #self.DPO_sell = bt.And(self.DPO.lines.DPO < 0, self.DPO.lines.DPO(-1) > 0)
        self.POS_buy = bt.And(self.POS.lines.POS > 80, self.POS.lines.POS(-1) < 80)
        self.POS_sell = bt.And(self.POS.lines.POS < 20, self.POS.lines.POS(-1) > 20)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.buyprice = order.executed.price

            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
#        if self.order:
#            return
        if self.POS_buy[0] and self.getposition(self.data).size == 0:
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.data.close[0], 100, self.getposition(self.data).size))
            self.order = self.buy(size=100)
        elif self.POS_sell[0] and self.getposition(self.data).size != 0:
            self.log('Sell CREATE (Close), Price: %.2f, Current Position: %i' % (self.data.close[0], self.getposition(self.data).size))
            self.order = self.close()         


## 单股回测

In [4]:
s_date = datetime.date(2021,12,31) - datetime.timedelta(days=365*2)
e_date = datetime.date(2021,12,31)
stock_code = '000166.SZ'
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'


if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(gf125, printlog=True) 


    # Create stock Data Feed
    if os.path.exists('.\\Data\\'+stock_code+'.csv'):
        df = pd.read_csv('.\\Data\\'+stock_code+'.csv', 
                    converters={'trade_date':lambda x:pd.to_datetime(x)}).set_index('trade_date')
    else:
        df = utilsJ.stock_tushare(token, stock_code, s_date, e_date)

    data = btfeed.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)

    # Set commission
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
52.44967320221249
48.98923164655962
52.74989869758554
50.29994347456997
57.8594484188634
55.83911241687638
54.234654009918806
53.23467737504879
49.47391613838269
50.21707536907349
46.736501656064085
35.26418923383239
23.74508458744334
25.85625998744635
26.987598115435812
9.369642484536362
0.0
0.0
0.0
0.0
0.0
2.88205869618506
0.0
1.3522912630272454
7.280429383362011
4.7019033198655285
0.0
5.0898591557982185
11.901884588434756
18.93413703223659
9.764397251573234
10.448948586770296
13.385062277071597
12.21008824223851
23.52727778638955
13.49052798896573
15.574313315977001
9.182281270626447
12.16579825328069
10.323352570285845
8.95591527124799
11.539543114401303
18.256473798411854
15.393478793329528
22.098182226450767
19.30463620603451
23.377775133617185
25.851005102232804
19.330517352386288
22.888911851681172
11.065563340531938
10.974818873062711
11.55863736459163
14.766052689446314
13.884463751269186
13.404987229840772
12.712254036187737
14.288906729268